# 3. Data Engineering

In [13]:
import modin.pandas as pd
import os
import numpy as np
import mysql.connector
from mysql.connector import errorcode

In [14]:
data_df = pd.read_csv('C:/Users/ASHA/BIGDATA/DATA/eccDNA_WGS_DATA.csv')

In [15]:
print(data_df.head(5))
print(data_df.shape)

     sample_id    chr  start_coord  end_coord  discordants  split_reads  \
0  SRR10704195  chr10     26935713   26940339            4            0   
1  SRR10704195  chr10     39506876   39506941            4            0   
2  SRR10704195  chr10     41860269   41860323           13            0   
3  SRR10704195  chr10    113689917  113689999            4            0   
4  SRR10704195  chr12     68786262   69204786           84           17   

   circle_score    mean_cov  stddev_cov  cov_incr_start  cov_incr_end  \
0          0.00    0.511457    0.724456        0.980000      0.839286   
1          0.00    2.553846    0.723895        0.882979      0.882979   
2          0.00  448.240741  177.763569        0.709262      0.968316   
3          0.00    0.902439    0.296720        0.986667      0.986667   
4        195.97   22.579384   11.965008        0.984020      0.998708   

   con_continuity  
0        0.609166  
1        0.000000  
2        0.000000  
3        0.097561  
4        0

In [16]:
# find all rows that fulfills your conditions and set class to 1

data_df['target'] = 0 # add a class column with 0 as default value

data_df.loc[(data_df['chr'] != 'chrM') & \
            (data_df['end_coord'] - data_df['start_coord'] > 20) & \
            (data_df['discordants'] > 1) & \
            (data_df['split_reads'] > 2) & \
            (abs(data_df['discordants'] - data_df['split_reads']) < 100) & \
            (data_df['circle_score'] > 50) & \
            (data_df['mean_cov'] > 1) & \
#            (data_df['mean_cov'] > data_df['stddev_cov']) & \
            (data_df['cov_incr_start'] > 0.33) & \
            (data_df['cov_incr_end'] > 0.33) & \
            (data_df['con_continuity'] < 0.7), "target"] = 1

In [17]:
data_df['target'].sum()

41

Confusion Matrix: A breakdown of predictions into a table showing correct predictions (the diagonal) and the types of incorrect predictions made (what classes incorrect predictions were assigned).
Precision: A measure of a classifiers exactness.
Recall: A measure of a classifiers completeness
F1 Score (or F-score): A weighted average of precision and recall.
I would also advice you to take a look at the following:

Kappa (or Cohen’s kappa): Classification accuracy normalized by the imbalance of the classes in the data.
ROC Curves: Like precision and recall, accuracy is divided into sensitivity and specificity and models can be chosen based on the balance thresholds of these values.

In [18]:
results_df = data_df.loc[data_df['target'] == 1]

In [19]:
results_df.head(15)

,sample_id,chr,start_coord,end_coord,discordants,split_reads,circle_score,mean_cov,stddev_cov,cov_incr_start,cov_incr_end,con_continuity,target
4,SRR10704195,chr12,68786262,69204786,84,17,195.97,22.579384,11.965008,0.984020,0.998708,0.010456,1
76,SRR10704196,chr4,52342742,54339698,56,10,82.93,12.839975,16.552357,0.981181,1.000000,0.360278,1
117,SRR10704197,chr12,68786261,69204786,66,16,160.00,35.299607,17.285185,0.996069,0.998639,0.009175,1
244,SRR10704199,chr12,68786263,69204786,40,10,79.99,18.272107,9.829682,0.958810,0.999009,0.009715,1
271,SRR10704199,chr4,52342742,54339698,55,8,84.81,9.957920,12.692801,0.922929,0.994073,0.324437,1
444,SRR10704202,chr12,68786262,69204786,50,9,72.91,18.732438,10.640702,0.981812,1.000000,0.012668,1
542,SRR10704203,chr12,68786262,69204786,12,5,56.00,19.194851,10.448780,0.996852,1.000000,0.011944,1
673,SRR10704204,chr12,68786262,69204786,10,5,71.98,18.228979,11.519119,0.984144,1.000000,0.013404,1
793,SRR10704205,chr12,68786262,69204786,12,7,79.96,18.547613,11.546354,0.997966,0.998680,0.014226,1
1117,SRR10704208,chr4,52342742,54339698,84,24,219.70,16.194016,21.646904,0.992505,0.999473,0.408322,1


In [20]:
results_df.to_csv(r"C:\Users\ASHA\BIGDATA\DATA\eccDNA_DATA-results.csv", index=False, header=True)

In [23]:
data_df.to_csv(r"C:\Users\ASHA\BIGDATA\DATA\eccDNA_WGS_DATA-TARGETS.csv", index=False, header=True)